<a href="https://colab.research.google.com/github/hemil19/online-data-science-ml-challenges/blob/master/MachineHack/Predict-The-Flight-Ticket-Price-Hackathon-master/Predicting_Flight_Ticket_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Science Process Workflow:**



1.   Understand the problem Statement
2.   Collect raw data needed for the problem
3.   Data Cleaning & preprocessing of raw data
4.   Explore the Data for insights
5.   Splitting Training,Validation & Testing Dataset 
6.   Data Modeling
7.   Choosing the best model based on Error Metric for Validation dataset
8.   Predicting for test dataset
9.   Submitting the test results 



**Importing Required Libraries**

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
train=pd.read_excel('/content/Data_Train.xlsx')
test=pd.read_excel('/content/Test_set.xlsx')
train.shape,test.shape

((10683, 11), (2671, 10))

In [3]:
train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


**Checking Null Values**

In [4]:
train.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [5]:
train[train['Total_Stops'].isnull()==True]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


**Handling Null Values**

In [6]:
train[(train['Source']=='Delhi') & (train['Destination']=='Cochin') & (train['Airline']=='Air India')]['Total_Stops'].value_counts(normalize=True)

2 stops     0.500000
1 stop      0.376676
non-stop    0.100536
3 stops     0.022788
Name: Total_Stops, dtype: float64

In [7]:
train[(train['Source']=='Delhi') & (train['Destination']=='Cochin') & (train['Airline']=='Air India')]['Route'].value_counts(normalize=True)

DEL → BOM → COK                0.156836
DEL → MAA → COK                0.103217
DEL → COK                      0.100536
DEL → BLR → COK                0.097855
DEL → AMD → BOM → COK          0.085791
DEL → GOI → BOM → COK          0.077748
DEL → HYD → BOM → COK          0.075067
DEL → HYD → MAA → COK          0.063003
DEL → CCU → BOM → COK          0.057641
DEL → JAI → BOM → COK          0.044236
DEL → BHO → BOM → COK          0.028150
DEL → RPR → NAG → BOM → COK    0.022788
DEL → LKO → BOM → COK          0.021448
DEL → JDH → BOM → COK          0.018767
DEL → TRV → COK                0.018767
DEL → UDR → BOM → COK          0.014745
DEL → NAG → BOM → COK          0.013405
Name: Route, dtype: float64

In [8]:
train.loc[9039,'Total_Stops']='2 stops'
train.loc[9039,'Route']='DEL → AMD → BOM → COK'

**Select duplicate rows except first occurrence based on all columns**

In [9]:
duplicateRowsDF = train[train.duplicated()]
print("Total Duplicate Rows except first occurrence based on all columns are :")
duplicateRowsDF.shape

Total Duplicate Rows except first occurrence based on all columns are :


(220, 11)

In [10]:
train.drop_duplicates(keep='first',inplace=True)

**Combining train and test dataset for faster pre-processing**

In [11]:
combine=train.append(test)
combine.reset_index(inplace=True)
combine.drop('index',axis=1,inplace=True)
combine.shape

(13134, 11)

In [12]:
combine['Depart_Time_Hour'] = pd.to_datetime(combine.Dep_Time).dt.hour
combine['Depart_Time_Minutes'] = pd.to_datetime(combine.Dep_Time).dt.minute
combine['Arr_Time_Hour'] = pd.to_datetime(combine.Arrival_Time).dt.hour
combine['Arr_Time_Minutes'] = pd.to_datetime(combine.Arrival_Time).dt.minute
combine.drop(['Route','Dep_Time','Arrival_Time'],axis=1,inplace=True)

In [13]:
combine['Total_Stops'].isnull().sum()

0

In [14]:
combine['Total_Stops']=combine['Total_Stops'].replace('non-stop','0 stops')

In [15]:
combine['Total_Stops']=[int(i[0]) for i in combine['Total_Stops'].values]

In [16]:
combine['Additional_Info']=combine['Additional_Info'].replace('No Info','No info')

In [17]:
combine['sourcedestination']=[(i,j) for i,j in zip(combine['Source'],combine['Destination'])]
combine.drop(['Source','Destination'],axis=1,inplace=True)

In [18]:
combine['Date_of_Journey']=pd.to_datetime(combine['Date_of_Journey'])

In [19]:
combine.head()

,Airline,Date_of_Journey,Duration,Total_Stops,Additional_Info,Price,Depart_Time_Hour,Depart_Time_Minutes,Arr_Time_Hour,Arr_Time_Minutes,sourcedestination
0,IndiGo,2019-03-24,2h 50m,0,No info,3897.0,22,20,1,10,"(Banglore, New Delhi)"
1,Air India,2019-01-05,7h 25m,2,No info,7662.0,5,50,13,15,"(Kolkata, Banglore)"
2,Jet Airways,2019-09-06,19h,2,No info,13882.0,9,25,4,25,"(Delhi, Cochin)"
3,IndiGo,2019-12-05,5h 25m,1,No info,6218.0,18,5,23,30,"(Kolkata, Banglore)"
4,IndiGo,2019-01-03,4h 45m,1,No info,13302.0,16,50,21,35,"(Banglore, New Delhi)"


In [20]:
combine.dtypes

Airline                        object
Date_of_Journey        datetime64[ns]
Duration                       object
Total_Stops                     int64
Additional_Info                object
Price                         float64
Depart_Time_Hour                int64
Depart_Time_Minutes             int64
Arr_Time_Hour                   int64
Arr_Time_Minutes                int64
sourcedestination              object
dtype: object

In [21]:
combine['Duration']=(pd.to_timedelta(combine['Duration']).dt.seconds // 60).astype(int)

In [22]:
combine['sourcedestination'].value_counts()

(Delhi, Cochin)          5491
(Kolkata, Banglore)      3570
(Banglore, Delhi)        1582
(Banglore, New Delhi)    1152
(Mumbai, Hyderabad)       883
(Chennai, Kolkata)        456
Name: sourcedestination, dtype: int64

In [23]:
combine['week'] = pd.to_datetime(combine['Date_of_Journey'], format='%y-%m-%d')
combine['Month'] = combine.week.dt.month
combine['Day'] = combine.week.dt.day
combine['Dayofweek'] = combine.week.dt.dayofweek

def get_weekend(row):
    if row.dayofweek == 5 or row.dayofweek == 6:
        return 1
    else:
        return 0
combine['IsWeekend'] = combine['week'].apply(get_weekend)
combine.drop(['Date_of_Journey','week'],axis=1,inplace=True)
combine.head()

,Airline,Duration,Total_Stops,Additional_Info,Price,Depart_Time_Hour,Depart_Time_Minutes,Arr_Time_Hour,Arr_Time_Minutes,sourcedestination,Month,Day,Dayofweek,IsWeekend
0,IndiGo,170,0,No info,3897.0,22,20,1,10,"(Banglore, New Delhi)",3,24,6,1
1,Air India,445,2,No info,7662.0,5,50,13,15,"(Kolkata, Banglore)",1,5,5,1
2,Jet Airways,1140,2,No info,13882.0,9,25,4,25,"(Delhi, Cochin)",9,6,4,0
3,IndiGo,325,1,No info,6218.0,18,5,23,30,"(Kolkata, Banglore)",12,5,3,0
4,IndiGo,285,1,No info,13302.0,16,50,21,35,"(Banglore, New Delhi)",1,3,3,0


In [24]:
combine["Airline"].replace(to_replace={'Multiple carriers Premium economy':'Other', 
                                                        'Jet Airways Business':'Other',
                                                        'Vistara Premium economy':'Other',
                                                        'Trujet':'Other'
                                                   },    
                                        inplace=True)
combine["Additional_Info"].replace(to_replace={'Change airports':'Other', 
                                                        'Business class':'Other',
                                                        '1 Short layover':'Other',
                                                        'Red-eye flight':'Other',
                                                        '2 Long layover':'Other',   
                                                   },    
                                        inplace=True)

In [25]:
combine=pd.get_dummies(combine)

In [26]:
X = combine[combine['Price'].isnull()!=True].drop(['Price'], axis=1)
y = combine[combine['Price'].isnull()!=True]['Price']

test = combine[combine['Price'].isnull()==True].drop(['Price'], axis=1)

X.shape, y.shape, test.shape

((10463, 30), (10463,), (2671, 30))

**Checking with LGBMRegressor**

In [27]:
err_lgm = []
y_pred_tot_lgm = []
features = X.columns
feature_importance_df = pd.DataFrame()

fold = KFold(n_splits=15)
i = 1
for train_index, test_index in fold.split(X, y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = LGBMRegressor(boosting_type='gbdt',
                       max_depth=5,
                       learning_rate=0.06,
                       n_estimators=6000,
                       min_child_weight=0.01,
                       colsample_bytree=0.5,
                       random_state=1994)
    m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=200,
          eval_metric='rmse',
          verbose=0)
    pred_y = m.predict(x_val)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = m.feature_importances_
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
    print(i, "err_lgm: ",1 - np.sqrt(np.square(np.log10(m.predict(x_val) +1) - np.log10(y_val +1)).mean()))
    err_lgm.append(1 - np.sqrt(np.square(np.log10(m.predict(x_val) +1) - np.log10(y_val +1)).mean()))
    pred_test = m.predict(test)
    i = i + 1
    y_pred_tot_lgm.append(pred_test)

1 err_lgm:  0.9450680769517392
2 err_lgm:  0.9471805454192046
3 err_lgm:  0.9427069904663192
4 err_lgm:  0.9363497944894088
5 err_lgm:  0.9390927987227735
6 err_lgm:  0.944390522729045
7 err_lgm:  0.9468109089899781
8 err_lgm:  0.9392666450463765
9 err_lgm:  0.9454737233694791
10 err_lgm:  0.9411586221700853
11 err_lgm:  0.9479899472199073
12 err_lgm:  0.9409586035007617
13 err_lgm:  0.9434786593143307
14 err_lgm:  0.9477307355962324
15 err_lgm:  0.9365961091372433


In [28]:
np.mean(err_lgm,0)

0.9429501788748591

In [29]:
err_xgb = []
y_pred_tot_xgb = []
features = X.columns
feature_importance_df = pd.DataFrame()

fold = KFold(n_splits=15)
i = 1
for train_index, test_index in fold.split(X, y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = XGBRegressor(boosting_type='gbdt',
                       max_depth=5,
                       learning_rate=0.05,
                       n_estimators=5000,
                       min_child_weight=0.01,
                       colsample_bytree=0.5,
                       random_state=1994)
    m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=200,
          eval_metric='rmse',
          verbose=0)
    pred_y = m.predict(x_val)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = m.feature_importances_
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
    print(i, "err_xgb: ",1 - np.sqrt(np.square(np.log10(m.predict(x_val) +1) - np.log10(y_val +1)).mean()))
    err_xgb.append(1 - np.sqrt(np.square(np.log10(m.predict(x_val) +1) - np.log10(y_val +1)).mean()))
    pred_test = m.predict(test)
    i = i + 1
    y_pred_tot_xgb.append(pred_test)

[20:37:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1 err_xgb:  0.9458898879466189
[20:38:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
2 err_xgb:  0.9491782466919668
[20:38:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
3 err_xgb:  0.9451714776113413
[20:39:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
4 err_xgb:  0.9409236317795004
[20:39:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
5 err_xgb:  0.9396957330877329
[20:39:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
6 err_xgb:  0.9472314088083696
[20:40:14] WARNING: /workspace/src/objective/regress

In [30]:
np.mean(err_xgb,0)

0.9446836805297213

**XGBRegressor gives good prediction as compared to LGBMRegressor**

In [31]:
submission=pd.read_excel('/content/Sample_submission.xlsx')
submission['Price']=np.mean(y_pred_tot_xgb,0)

In [32]:
submission['Price'].head()

0    14048.060547
1     4842.430176
2    12502.802734
3    10927.246094
4     3453.536865
Name: Price, dtype: float32

In [33]:
submission.to_csv('xgb_average.csv',index=False)